In [1]:
# Learning with Group Privacy
from MLModel import *

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset

import torch
import numpy as np
import os

import time

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [2]:
scattering, K, (h, w) = get_scatter_transform()
scattering.to(device)

def get_scattered_feature(dataset):
    scatters = []
    targets = []
    
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=256, shuffle=True, num_workers=1, pin_memory=True)

    
    for (data, target) in loader:
        data, target = data.to(device), target.to(device)
        if scattering is not None:
            data = scattering(data)
        scatters.append(data)
        targets.append(target)

    scatters = torch.cat(scatters, axis=0)
    targets = torch.cat(targets, axis=0)

    data = torch.utils.data.TensorDataset(scatters, targets)
    return data

def load_mnist():
    train = datasets.FashionMNIST(root="~/data/", train=True, download=True, transform=transforms.ToTensor())
    test = datasets.FashionMNIST(root="~/data/", train=False, download=True, transform=transforms.ToTensor())
    
    # get scattered features
    train = get_scattered_feature(train)
    test = get_scattered_feature(test)
    
    train_data = train[:][0].squeeze().cpu().float()
    train_label = train[:][1].cpu()
    
    test_data = test[:][0].squeeze().cpu().float()
    test_label = test[:][1].cpu()

    dataset = [(train_data, train_label), (test_data, test_label)]
    return dataset

In [3]:
d = load_mnist()

train_data = torch.tensor(d[0][0]).to(device)
train_label = torch.tensor(d[0][1]).to(device)

test_data = torch.tensor(d[1][0]).to(device)
test_label = torch.tensor(d[1][1]).to(device)

data_size = len(train_label)
print("#. training records=",data_size)

torch.cuda.empty_cache()

/tmp/ipykernel_2321299/2464145456.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor(d[0][0]).to(device)
/tmp/ipykernel_2321299/2464145456.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_label = torch.tensor(d[0][1]).to(device)
/tmp/ipykernel_2321299/2464145456.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(d[1][0]).to(device)
/tmp/ipykernel_2321299/2464145456.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

#. training records= 60000


In [4]:
"""
FL model parameters.
"""
import warnings
warnings.filterwarnings("ignore")

lr = 0.2

configs = {
    'output_size': 10,
    'data_size': data_size,
    'model': 'scatter',
    'data': d,
    'lr': lr,
    'E': 500,
    'delta': 1e-5,
    'q': 0.05,
    'clip': 0.1,
    'batch_size': 128,
    'device': device
}

In [5]:
def test_acc(model):
    model.eval()
    correct = 0
    tot_sample = len(test_data)
    #for i in range(len(test_data)):
    
    t_pred_y = model(test_data)
    _, predicted = torch.max(t_pred_y, 1)
    correct += (predicted == test_label).sum().item()
    
    acc = correct / tot_sample
    return acc

def train():
    model.train()
    criterion = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.SGD(model.parameters(), lr=configs['lr'], momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters())

    acc = []
    start_time = time.time()
    for e in range(configs['E']):
        # randomly select q fraction samples from dataset by poisson sampling
        idx = np.where(np.random.rand(len(torch_train[:][0])) < configs['q'])[0]

        sampled_dataset = TensorDataset(torch_train[idx][0], torch_train[idx][1])
        sample_data_loader = DataLoader(
            dataset=sampled_dataset,
            batch_size=configs['batch_size'],
            shuffle=True
        )

        optimizer.zero_grad()

        clipped_grads = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        for batch_x, batch_y in sample_data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            pred_y = model(batch_x.float())
            loss = criterion(pred_y, batch_y.long())

            # bound l2 sensitivity (gradient clipping)
            # clip each of the gradient in subset
            for i in range(loss.size()[0]):
                loss[i].backward(retain_graph=True)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=configs['clip'])
                for name, param in model.named_parameters():
                    clipped_grads[name] += param.grad 
                model.zero_grad()

        # add Gaussian noise
        for name, param in model.named_parameters():
            clipped_grads[name] += gaussian_noise(clipped_grads[name].shape, configs['clip'], configs['sigma'], device=device)

        # scale back
        for name, param in model.named_parameters():
            clipped_grads[name] /= (configs['data_size']*configs['q'])

        for name, param in model.named_parameters():
            param.grad = clipped_grads[name]

        # update local model
        optimizer.step()
        
        if (e+1)%500 == 0:
            acc = test_acc(model)
            print("iters = {:d}, acc = {:.4f}".format(e+1, acc), " Time taken: %.2fs" % (time.time() - start_time))
    return acc

# Varying group size

In [6]:
"""
noise scale is calibrated using binary search,
see 'calibrate_sgm_noise.ipynb' file
"""
m_list = [8, 16, 32, 64]

noise_rdp= [19.09490966796875, 46.65306091308594, 114.18656921386719, 279.62950134277344]
noise_ours= [19.520643949508667, 31.144098043441772, 52.82178044319153, 94.88274216651917]

### Naive RDP

In [7]:
for sigma in noise_rdp:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(81, (7, 7), input_norm="GroupNorm", num_groups=81).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

sigma = 19.09490966796875
iters = 500, acc = 0.8331  Time taken: 530.71s
iters = 500, acc = 0.8337  Time taken: 532.15s
iters = 500, acc = 0.8359  Time taken: 531.46s
iters = 500, acc = 0.8326  Time taken: 528.40s
iters = 500, acc = 0.8362  Time taken: 529.68s
avg acc = 0.8343
sigma = 46.65306091308594
iters = 500, acc = 0.8238  Time taken: 544.61s
iters = 500, acc = 0.8201  Time taken: 560.29s
iters = 500, acc = 0.8213  Time taken: 571.26s
iters = 500, acc = 0.8234  Time taken: 566.83s
iters = 500, acc = 0.8225  Time taken: 534.71s
avg acc = 0.82222
sigma = 114.18656921386719
iters = 500, acc = 0.7653  Time taken: 540.69s
iters = 500, acc = 0.7642  Time taken: 610.75s
iters = 500, acc = 0.7698  Time taken: 587.65s
iters = 500, acc = 0.7654  Time taken: 608.78s
iters = 500, acc = 0.7641  Time taken: 602.67s
avg acc = 0.76576
sigma = 279.62950134277344
iters = 500, acc = 0.6484  Time taken: 606.35s
iters = 500, acc = 0.6075  Time taken: 578.34s
iters = 500, acc = 0.6096  Time taken: 535

### Ours

In [8]:
for sigma in noise_ours:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(81, (7, 7), input_norm="GroupNorm", num_groups=81).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

sigma = 19.520643949508667
iters = 500, acc = 0.8305  Time taken: 548.06s
iters = 500, acc = 0.8369  Time taken: 528.79s
iters = 500, acc = 0.8361  Time taken: 649.17s
iters = 500, acc = 0.8357  Time taken: 1697.89s
iters = 500, acc = 0.8338  Time taken: 536.41s
avg acc = 0.8346
sigma = 31.144098043441772
iters = 500, acc = 0.8315  Time taken: 533.71s
iters = 500, acc = 0.8262  Time taken: 539.53s
iters = 500, acc = 0.8320  Time taken: 542.28s
iters = 500, acc = 0.8286  Time taken: 536.33s
iters = 500, acc = 0.8290  Time taken: 542.74s
avg acc = 0.8294599999999999
sigma = 52.82178044319153
iters = 500, acc = 0.8197  Time taken: 553.88s
iters = 500, acc = 0.8119  Time taken: 542.46s
iters = 500, acc = 0.8178  Time taken: 527.32s
iters = 500, acc = 0.8198  Time taken: 539.66s
iters = 500, acc = 0.8145  Time taken: 549.01s
avg acc = 0.8167399999999999
sigma = 94.88274216651917
iters = 500, acc = 0.7752  Time taken: 536.70s
iters = 500, acc = 0.7860  Time taken: 532.26s
iters = 500, acc = 

## Varying privacy parameter $\epsilon$

In [9]:
"""
noise scale is calibrated using binary search,
see 'calibrate_sgm_noise.ipynb' file
"""
# m = 32
eps_list = [8, 7, 6, 5, 4, 3, 2]

noise_rdp= [62.96098327636719, 70.53829956054688, 80.5775146484375, 94.09147644042969, 114.18656921386719, 147.5915069580078, 211.9739990234375]
noise_ours= [29.204277992248535, 32.63718247413635, 37.39244818687439, 43.51409435272217, 52.82178044319153, 68.23326706886292, 97.94802665710449]

### Naive RDP

In [10]:
for sigma in noise_rdp:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(81, (7, 7), input_norm="GroupNorm", num_groups=81).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

sigma = 62.96098327636719
iters = 500, acc = 0.8139  Time taken: 532.04s
iters = 500, acc = 0.8136  Time taken: 528.60s
iters = 500, acc = 0.8071  Time taken: 527.33s
iters = 500, acc = 0.8131  Time taken: 564.92s
iters = 500, acc = 0.8091  Time taken: 537.66s
avg acc = 0.81136
sigma = 70.53829956054688
iters = 500, acc = 0.8049  Time taken: 531.37s
iters = 500, acc = 0.8034  Time taken: 532.73s
iters = 500, acc = 0.8056  Time taken: 538.73s
iters = 500, acc = 0.8011  Time taken: 530.53s
iters = 500, acc = 0.8029  Time taken: 529.56s
avg acc = 0.80358
sigma = 80.5775146484375
iters = 500, acc = 0.7942  Time taken: 528.55s
iters = 500, acc = 0.7988  Time taken: 532.27s
iters = 500, acc = 0.7975  Time taken: 529.44s
iters = 500, acc = 0.7974  Time taken: 531.10s
iters = 500, acc = 0.7985  Time taken: 533.55s
avg acc = 0.79728
sigma = 94.09147644042969
iters = 500, acc = 0.7802  Time taken: 533.54s
iters = 500, acc = 0.7833  Time taken: 528.94s
iters = 500, acc = 0.7887  Time taken: 530.0

### Ours

In [ ]:
for sigma in noise_ours:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(81, (7, 7), input_norm="GroupNorm", num_groups=81).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

sigma = 29.204277992248535
iters = 500, acc = 0.8284  Time taken: 1755.45s
iters = 500, acc = 0.8328  Time taken: 573.85s
iters = 500, acc = 0.8305  Time taken: 572.50s
iters = 500, acc = 0.8296  Time taken: 582.63s
iters = 500, acc = 0.8316  Time taken: 572.12s
avg acc = 0.83058
sigma = 32.63718247413635
iters = 500, acc = 0.8295  Time taken: 573.74s
iters = 500, acc = 0.8330  Time taken: 574.27s
iters = 500, acc = 0.8259  Time taken: 573.11s
iters = 500, acc = 0.8297  Time taken: 585.12s
iters = 500, acc = 0.8272  Time taken: 583.53s
avg acc = 0.8290599999999999
sigma = 37.39244818687439
iters = 500, acc = 0.8300  Time taken: 571.59s
iters = 500, acc = 0.8295  Time taken: 572.04s
iters = 500, acc = 0.8293  Time taken: 578.68s
iters = 500, acc = 0.8278  Time taken: 602.71s
iters = 500, acc = 0.8254  Time taken: 584.43s
avg acc = 0.8283999999999999
sigma = 43.51409435272217
iters = 500, acc = 0.8263  Time taken: 607.63s
iters = 500, acc = 0.8169  Time taken: 573.30s
iters = 500, acc = 